In [41]:
s = 'ABCD'
s2 = '-asdfABCD'
s3 = '#'
s4 = 'abcd-'
s5 = 'ABD-'

In [2]:
import re

In [51]:
print(re.match('[A-Z]*[a-z]+|-[a-z][A-Z]*-*', s2))

<re.Match object; span=(0, 2), match='-a'>


In [61]:
for elem in [s, s2, s3, s4, s5]:
    print(re.match('[A-Z]*[a-z]*|-[a-z][A-Z]*-*', elem))

<re.Match object; span=(0, 4), match='ABCD'>
<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(0, 4), match='abcd'>
<re.Match object; span=(0, 3), match='ABD'>


In [69]:
for elem in [s, s2, s3, s4, s5]:
    print(re.match('^[A-Z]+|^-|^[a-z]', elem))

<re.Match object; span=(0, 4), match='ABCD'>
<re.Match object; span=(0, 1), match='-'>
None
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 3), match='ABD'>


In [73]:
for elem in [s, s2, s3, s4, s5]:
    print(re.match('[^#]', elem))

<re.Match object; span=(0, 1), match='A'>
<re.Match object; span=(0, 1), match='-'>
None
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='A'>
